In [1]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

### Подключаемся к Google Sheets

In [18]:
# Define the scope and credentials for accessing Google Sheets
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/a1111/Documents/Notebooks/work-key.json', scope)

# Authenticate and open the Google Sheets file
gc = gspread.authorize(credentials)
# You can change the name of the sheet you want to open below
spreadsheet = gc.open('OMMAS-176 Купонные статьи') 

#### Don't forget to give this service account a permission to edit the sheet
gsheet-reader@project-id-0565238085837732033.iam.gserviceaccount.com

### Функция ниже объединяет два DataFrame (df_2022 и df_2023) по заданным столбцам (join_columns) и создает новый DataFrame, содержащий значения метрик (metrics) для каждого года. 


In [2]:
def merge_and_create_yoy(df_2022, df_2023, join_columns, metrics):
# Первый сценарий

# В начале проверяем пуст ли список колонок для объединения, и если да, то выполняем код ниже    
    if len(join_columns) == 0:
        
        # Step 1: In case if we have similar values in the first column we change one of the values, so we won't get an error
        first_col_name = df_2022.columns[0]
        df_2022.at[0, first_col_name] = 2020
        
        # Step 2: Merge two dataframes grouping by the column 'action_year'
        merged_df = pd.concat([df_2022, df_2023]).groupby(['action_year' if 'action_year' in df_2022.columns else 'action_month']).sum().reset_index()

        # Step 2: Convert the values in merged_df to numeric type
        merged_df = merged_df.apply(pd.to_numeric, errors='coerce')

        # Step 3: Check for orders_mobile column
        if 'orders_mobile' in metrics:
            # Step 1: Convert the values in 'orders_mobile' and 'count_orders' columns to numeric type
            merged_df.loc[0:1, 'orders_mobile'] = pd.to_numeric(merged_df.loc[0:1, 'orders_mobile'])
            merged_df.loc[0:1, 'count_orders'] = pd.to_numeric(merged_df.loc[0:1, 'count_orders'])

            # Step 2: Add the new column 'mobile_orders_%'
            merged_df.loc[0:1, 'mobile_orders_%'] = (merged_df.loc[0:1, 'orders_mobile'] / merged_df.loc[0:1, 'count_orders']) * 100

            # Step 3: Drop the 'orders_mobile' column from merged_df
            merged_df = merged_df.drop('orders_mobile', axis=1)

        # Step 4: Check for 'avg_check' column
        if 'avg_check' in metrics:
            # Step 1: Convert the values in 'gmv' and 'count_orders' columns to numeric type
            merged_df.loc[0:1, 'gmv'] = pd.to_numeric(merged_df.loc[0:1, 'gmv'])
            merged_df.loc[0:1, 'count_orders'] = pd.to_numeric(merged_df.loc[0:1, 'count_orders'])

            # Step 2: Add the new column 'avg_check'
            merged_df.loc[0:1, 'avg_check'] = merged_df.loc[0:1, 'gmv'] / merged_df.loc[0:1, 'count_orders']

        # Step 5: Calculate the year-over-year percentage change
        percentage_change = ((merged_df.loc[1] / merged_df.loc[0]) - 1) * 100
        percentage_change = percentage_change.round(2)

        # Step 6: Add a new row to the index of merged_df
        merged_df.loc['%'] = percentage_change

        # Step 7: Append the percentage sign to each value in the '%' index of merged_df
        merged_df.loc['%'] = merged_df.loc['%'].astype(str) + '%'

        # Step 8: remove the first value in the '%' index of merged_df and a row value in the mobile_orders_%
        first_column_name = merged_df.columns[0]
        merged_df.at['%', first_column_name] = '%'
        merged_df.at[0, first_column_name] = df_1_suffix
        merged_df.at[1, first_column_name] = df_2_suffix
        if 'orders_mobile' in metrics:
            merged_df.loc['%', 'mobile_orders_%'] = 'NaN'
            
        # Step 9: If we have 'kind' and 'subkind' values we assign them correct values
        if 'kind' in df_2022.columns:
            merged_df.loc[[0, 1], 'kind'] = df_2022['kind'].iloc[0]
            merged_df.loc['%', 'kind'] = '-'
            
        if 'subkind' in df_2022.columns:
            merged_df.loc[[0, 1], 'subkind'] = df_2022['subkind'].iloc[0]
            merged_df.loc['%', 'subkind'] = '-'

        # Step 10: If we have 'offer_category' and 'offer_subcategory' values we assign them correct values
        if 'offer_category' in df_2022.columns:
            merged_df.loc[[0, 1], 'offer_category'] = df_2022['offer_category'].iloc[0]
            merged_df.loc['%', 'offer_category'] = '-'
            
        if 'offer_subcategory' in df_2022.columns:
            merged_df.loc[[0, 1], 'offer_subcategory'] = df_2022['offer_subcategory'].iloc[0]
            merged_df.loc['%', 'offer_subcategory'] = '-'

        # Step 11: If we have 'ali_category' values we assign them correct values
        if 'ali_category' in df_2022.columns:
            merged_df.loc[[0, 1], 'ali_category'] = df_2022['ali_category'].iloc[0]
            merged_df.loc['%', 'ali_category'] = '-'
            
        #   Наконец, функция возвращает итоговый DataFrame. 
        return merged_df
    
# Второй сценарий

#   Если же список колонок для объединения не пуст, то происходит объединение двух DataFrame с помощью метода merge().  
    df = pd.merge(df_2022, df_2023, on=join_columns, how='outer', suffixes=(df_1_suffix, df_2_suffix))
    df.fillna(0, inplace=True)
    
# Если в metrics есть колонка avg_check, то вычислим средний чек.  
    if 'avg_check' in metrics:
        df[f'avg_check{df_1_suffix}'] = round(df[f'gmv{df_1_suffix}'] / df[f'count_orders{df_1_suffix}'], 2)
        df[f'avg_check{df_2_suffix}'] = round(df[f'gmv{df_2_suffix}'] / df[f'count_orders{df_2_suffix}'], 2)
        df['avg_check_%'] = (df[f'avg_check{df_2_suffix}'] / df[f'avg_check{df_1_suffix}'] - 1).apply(lambda x: f'{round(x * 100, 2)}%')
        
#   Далее, из объединенного DataFrame выбираются только нужные столбцы, включая столбцы с метриками для каждого года.
#   Столбцы, которые будут выбраны, состоят из `join_columns` (переменная, содержащая список столбцов для объединения) 
#   и столбцов, которые имеют названия, состоящие из исходных названий столбцов `metrics`, 
#   с добавлением суффиксов "_2022" и "_2023".         
    df = df[[*join_columns,
             *[f'{col}{df_1_suffix}' for col in metrics],
             *[f'{col}{df_2_suffix}' for col in metrics]]]

#   Затем для каждой метрики вычисляется год к году (YoY) изменение в процентах и добавляется новый столбец с этими значениями. 
    for col in metrics:
        col_2022 = f'{col}{df_1_suffix}'
        col_2023 = f'{col}{df_2_suffix}'
        col_YoY = f'{col}_YoY'
        df[col_YoY] = round((df[col_2023] / df[col_2022] - 1) * 100, 2).astype(str) + '%'
        
#   Затем столбцы с метриками для каждого года, а также столбцы с YoY изменениями сортируются и добавляются в итоговый DataFrame.         
    sorted_columns = []

    for col in metrics:
        sorted_columns.append(f'{col}{df_1_suffix}')
        sorted_columns.append(f'{col}{df_2_suffix}')
        sorted_columns.append(f'{col}_YoY')
    
    df = df.reindex(columns=join_columns+sorted_columns)
    
#   Если в metrics есть колонка orders_mobile, то вычислим долю мобильных заказов и удалим ненужный столбец orders_mobile_YoY
    if 'orders_mobile' in metrics:
        df[f'orders_mobile{df_1_suffix}'] = round((df[f'orders_mobile{df_1_suffix}'] / df[f'count_orders{df_1_suffix}']) * 100, 2).astype(str) + '%'
        df[f'orders_mobile{df_2_suffix}'] = round((df[f'orders_mobile{df_2_suffix}'] / df[f'count_orders{df_2_suffix}']) * 100, 2).astype(str) + '%'    
        df = df.drop('orders_mobile_YoY', axis=1)
        
#   Наконец, функция возвращает итоговый DataFrame.    
    return df

# Ниже введем аргументы для функции

In [4]:
# Укажем путь к файлам
df_2022 = pd.read_csv('/Users/a1111/Downloads/2022 by categories.csv', sep=',')
df_2023 = pd.read_csv('/Users/a1111/Downloads/2022 by categories.csv', sep=',')
df_2022.head(5)

,action_month,ali_category,count_orders,gmv
0,2022-10-01,320,756,21837.63
1,2022-12-01,200000343,7950,99453.51
2,2022-08-01,200000532,2139,62513.97
3,2022-05-01,44,39691,1644187.08
4,2022-07-01,13,13738,563019.59


In [360]:
#   Если нужны будут другие суффиксы для столбцов то их можно изменить ниже.
df_1_suffix = '_feb' 
df_2_suffix = '_mar'

# `join_columns` (переменная, содержащая список столбцов для объединения) если оставить её пустой,
# то отработает скрипт для однострочных данных
# я оставлю названия столбцов, которые можно будет copy/paste в переменную
# 'action_country', 'ali_category', 'kind', 'subkind', 'offer_category', 'offer_subcategory'
join_columns = ['action_country', 'kind', 'subkind']

# `metrics` (переменная, содержащая список метрик)
# я оставлю названия метрик, которые можно будет copy/paste в переменную
# 'count_orders', 'gmv','aov', 'active_publishers', 'publisher_payment', 'active_offers', 'orders_mobile', 'ratio_percentage', 'ratio'
metrics = ['count_orders', 'ratio_percentage']

df = merge_and_create_yoy(df_2022, df_2023, join_columns, metrics)

df.head()

,action_country,kind,subkind,count_orders_feb,count_orders_mar,count_orders_YoY,ratio_percentage_feb,ratio_percentage_mar,ratio_percentage_YoY
0,Brazil,social_network,social_network_facebook,563.0,989.0,75.67%,1.2300,1.7100,39.02%
1,Brazil,drop_shipping,drop_shipping_topdser,63.0,71.0,12.7%,0.1380,0.1230,-10.87%
2,Brazil,applications,mobile_app,623.0,447.0,-28.25%,1.3600,0.7710,-43.31%
3,Brazil,web_service,coupons,2758.0,3576.0,29.66%,6.0400,6.1700,2.15%
4,Brazil,applications,browser,19.0,17.0,-10.53%,0.0416,0.0293,-29.57%


## Отправляем финальный файл в Google sheets


In [340]:
worksheet = spreadsheet.add_worksheet(title=f'{df_1_suffix} vs {df_2_suffix}', rows=100, cols=20)
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

/var/folders/g4/6cm6j5wd459g4ygf60jcky4r0000gn/T/ipykernel_17814/4120570201.py:2: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '1lztd9jWCiTN7qS6L-YtpqTGUKLi3pnwId9xeuVLJeb4',
 'updatedRange': "'_feb vs _mar'!A1:I318",
 'updatedRows': 318,
 'updatedColumns': 9,
 'updatedCells': 2862}

## Выгружаем финальный файл

In [361]:
df.to_csv('task_file.csv', index=False)

### Опционально

In [357]:
# Добавим вычисление доли от общего для каждого ряда определенного столбца
# Просто вписать нужные столбцы в переменную ratio_column
ratio_column = ['count_orders']

for col in ratio_column:
    col_2022 = f'{col}' + df_1_suffix
    col_2023 = f'{col}' + df_2_suffix
    col_ratio_2022 = f'{col_2022}_ratio'
    col_ratio_2023 = f'{col_2023}_ratio'
    df[col_ratio_2022] = (df[col_2022] / df[col_2022].sum() * 100).apply(lambda x: f'{round(x, 2)}%')
    df[col_ratio_2023] = (df[col_2023] / df[col_2023].sum() * 100).apply(lambda x: f'{round(x, 2)}%')
    
df.head()

,kind,subkind,count_orders_feb,count_orders_mar,count_orders_YoY,count_orders_feb_ratio,count_orders_mar_ratio
0,web_service,cashback,375483,439384.0,17.02%,23.81%,25.68%
1,website,other,338285,365594.0,8.07%,21.45%,21.36%
2,website,affiliate_store,221982,218141.0,-1.73%,14.08%,12.75%
3,web_service,coupons,160813,188024.0,16.92%,10.2%,10.99%
4,traffic_purchase,traffic_purchase_google_ads,73476,69490.0,-5.42%,4.66%,4.06%


## Отправляем финальный файл в Google sheets


In [358]:
worksheet = spreadsheet.add_worksheet(title=f'{df_1_suffix} vs {df_2_suffix}', rows=100, cols=20)
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

/var/folders/g4/6cm6j5wd459g4ygf60jcky4r0000gn/T/ipykernel_17814/4120570201.py:2: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '1lztd9jWCiTN7qS6L-YtpqTGUKLi3pnwId9xeuVLJeb4',
 'updatedRange': "'_feb vs _mar'!A1:G58",
 'updatedRows': 58,
 'updatedColumns': 7,
 'updatedCells': 406}

## Выгружаем финальный файл на устройство

In [10]:
df.to_csv('task_file.csv', index=False)

In [20]:
df = pd.read_csv('/Users/a1111/Downloads/10 no ali.csv', sep=',')
df.head(5)

,action_month,offer_category,offer_subcategory,count_orders,gmv,aov
0,jan-jun 24,Financial Programs,Deposits,302,0.0,0.0
1,jan-jun 24,Financial Programs,Investments,46,0.0,0.0
2,jan-jun 24,SME,NaN,5,0.0,0.0
3,jan-jun 24,Online Games,Client Games,10437,0.0,0.0
4,jan-jun 24,Mobile Programs,Mobile Services,21,0.0,0.0


In [17]:
# Для AliExpress меняем названия категорий с чисел на слова
df['ali_category'] = df['ali_category'] \
        .replace({2: 'Food',
                 3: 'Apparel & Accessories',
                 6: 'Home Appliances',
                 7: 'Computer & Office',
                 13: 'Home Improvement',
                 15: 'Home & Garden',
                 18: 'Sports & Entertainment',
                 21: 'Office & School Supplies',
                 26: 'Toys & Hobbies',
                 30: 'Security & Protection',
                 34: 'Automobiles, Parts & Accessories',
                 36: 'Jewelry & Accessories',
                 39: 'Lights & Lighting',
                 44: 'Consumer Electronics',
                 66: 'Beauty & Health',
                 320: 'Weddings & Events',
                 322: 'Shoes',
                 348: 'Shirts',
                 349: 'Skirts',
                 502: 'Electronic Components & Supplies',
                 504: 'Electronic Data Systems',
                 509: 'Phones & Telecommunications',
                 515: 'Electronics Stocks',
                 701: 'Desktops',
                 702: 'Laptops',
                 703: 'Servers',
                 1420: 'Tools',
                 1428: 'Abrasives',
                 1501: 'Mother & Kids',
                 1503: 'Furniture',
                 1511: 'Watches',
                 1524: 'Luggage & Bags',
                 2202: 'Books',
                 2209: 'Map',
                 2213: 'Magazines',
                 2621: 'Action & Toy Figures',
                 3712: 'Furniture Accessories',
                 4002: 'Electronics Production Machinery',
                 4204: 'Abrasive Tools',
                 4338: 'Woodworking Machinery Parts',
                 32005: 'Wedding Dresses',
                 33906: 'Scarf, Hat & Glove Sets',
                 50906: 'Walkie Talkie',
                 70806: 'Computer Cables & Connectors',
                 142001: 'Tool Parts',
                 150403: 'Holiday Lighting',
                 150407: 'Electronic Signs',
                 150412: 'EL Products',
                 152401: 'Backpacks',
                 152402: 'Briefcases',
                 152409: 'Bag Parts & Accessories',
                 211110: 'Filing Products',
                 380650: 'Waist Packs',
                 708022: 'Computer Cleaners',
                 3280111: 'Suspenders',
                 5090301: 'Mobile Phones',
                 39050508: 'Night Lights',
                 70803003: 'Mini PC',
                 100001793: 'Diecasts & Toy Vehicles',
                 100002856: 'Handbags',
                 100003177: 'File Folder Accessories',
                 100005329: 'KVM Switches',
                 100005330: 'Industrial Computer & Accessories',
                 100006468: 'Drill Bit',
                 100006471: 'Saw Blade',
                 100007488: 'Power Tool Accessories',
                 200000126: 'Pocket & Fob Watches',
                 200000181: 'Jewelry Findings & Components',
                 200000297: 'Apparel Accessories',
                 200000298: 'Belts',
                 200000305: 'Ties',
                 200000306: 'Cummerbunds',
                 200000343: "Men's Clothing",
                 200000344: 'Hoodies & Sweatshirts',
                 200000345: "Women's Clothing",
                 200000346: 'Blouses & Shirts',
                 200000347: 'Dresses',
                 200000348: 'Hoodies & Sweatshirts',
                 200000361: 'Jeans',
                 200000362: 'Jumpsuits, Playsuits & Bodysuits',
                 200000366: 'Pants & Capris',
                 200000367: 'Shorts',
                 200000373: 'Sweaters',
                 200000378: 'Jeans',
                 200000380: 'Pants',
                 200000384: "Men's Socks",
                 200000392: 'Arm Warmers',
                 200000393: 'Earmuffs',
                 200000394: 'Gloves & Mittens',
                 200000395: 'Headwear',
                 200000399: 'Scarves & Wraps',
                 200000400: 'Handkerchiefs',
                 200000523: 'Travel Accessories',
                 200000532: 'Novelty & Special Use',
                 200000865: 'Leggings',
                 200001067: 'Novelty Lighting',
                 200001079: 'Additional Pay on Your Order',
                 200001086: 'Tablets',
                 200001478: 'Jewelry Tools & Equipments',
                 200001479: 'Jewelry Packaging & Display',
                 200001621: 'Giveaways',
                 200001791: 'First Aid Kits',
                 200003494: "Women's Sets",
                 200003495: "Men's Sets",
                 200004218: 'Auto Sale',
                 200004276: 'Stationery Sticker',
                 200004343: 'Transmission & Cables',
                 200004358: 'Test',
                 200165144: 'Hair Extensions & Wigs',
                 200183146: 'Riveter Guns',
                 200295142: 'Mask',
                 200295143: 'Knee Sleeve',
                 200301150: 'Belt Buckle',
                 200326144: 'Special Engineering Lighting',
                 200333142: 'Drawstring Bags',
                 200346143: 'Collar Stays',
                 200362143: 'Couple Watches',
                 200362144: "Children's Watches",
                 200370154: 'Smart Jewelry',
                 200376142: 'Custom-made-Charge',
                 200380144: 'Walkie Talkie Parts & Accessories',
                 200392144: 'Pocket Squares',
                 200574005: 'Underwear',
                 200726001: 'Wallpaper Sample',
                 201161809: 'Fashionable Canes',
                 201169612: 'Virtual Products',
                 201355758: 'Motorcycle Equipments & Parts',
                 200001075: 'Special Category'})
df.head()

,action_month,ali_category,count_orders,gmv,aov
0,jan-jun 24,Furniture,1824,164168,90.0
1,jan-jun 24,Weddings & Events,1519,88739,58.4
2,jan-jun 24,Hair Extensions & Wigs,3465,139687,40.3
3,jan-jun 24,Security & Protection,10279,265830,25.9
4,jan-jun 24,Home Appliances,12770,308230,24.1


## Отправляем финальный файл в Google sheets


In [19]:
worksheet = spreadsheet.add_worksheet(title='data', rows=100, cols=20)
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

/var/folders/g4/6cm6j5wd459g4ygf60jcky4r0000gn/T/ipykernel_50950/1577828394.py:2: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '1VCCuWCouJMXSh-WBhA1-mEpLAUbjFmSHcP8_DtWQVW4',
 'updatedRange': 'data!A1:E33',
 'updatedRows': 33,
 'updatedColumns': 5,
 'updatedCells': 165}

## Выгружаем финальный файл

In [354]:
df.to_csv('task_file.csv', index=False)